In [1]:
# !pip install tensorflow-datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
# from keras.models import load_model
from datetime import datetime
tf.enable_eager_execution()

C:\Users\Affinity\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
W1024 13:34:46.843391 22000 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)             

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).batch(64) # 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [10]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_acc',
    save_best_only=True)

In [11]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [13]:
history = model.fit(train_data,
                    epochs=30,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 13:37:29.704781: {'loss': 1.6441709135055542, 'acc': 0.458725, 'val_loss': 1.385859111691736, 'val_acc': 0.5181}
Epoch 1 ends at 13:39:13.295584: {'loss': 1.2065217227935792, 'acc': 0.583425, 'val_loss': 1.4675401604858933, 'val_acc': 0.5116}
Epoch 2 ends at 13:41:04.977734: {'loss': 1.0747028533935548, 'acc': 0.6258, 'val_loss': 1.1032659381058565, 'val_acc': 0.6153}
Epoch 3 ends at 13:42:57.544480: {'loss': 1.0071527797698974, 'acc': 0.647075, 'val_loss': 1.1248166731968048, 'val_acc': 0.6146}
Epoch 4 ends at 13:44:39.334103: {'loss': 0.9662142958641052, 'acc': 0.6618, 'val_loss': 0.9696164579148505, 'val_acc': 0.6648}
Epoch 5 ends at 13:46:20.134371: {'loss': 0.9272185616493225, 'acc': 0.674825, 'val_loss': 1.1348700648660113, 'val_acc': 0.619}
Epoch 6 ends at 13:48:05.656005: {'loss': 0.9022877049446106, 'acc': 0.68645, 'val_loss': 0.9782326680839442, 'val_acc': 0.6628}
Epoch 7 ends at 13:49:48.526734: {'loss': 0.8827059274673462, 'acc': 0.693175, 'val_loss': 0.9993

In [14]:
model.evaluate(test_data)

    157/Unknown - 0s 33ms/step - loss: 1.0632 - acc: 0.718 - 0s 26ms/step - loss: 1.0001 - acc: 0.734 - 0s 24ms/step - loss: 1.0158 - acc: 0.718 - 0s 23ms/step - loss: 0.9576 - acc: 0.734 - 0s 22ms/step - loss: 0.9502 - acc: 0.721 - 0s 22ms/step - loss: 0.9402 - acc: 0.713 - 0s 22ms/step - loss: 0.9489 - acc: 0.705 - 0s 22ms/step - loss: 0.9537 - acc: 0.707 - 0s 22ms/step - loss: 0.9659 - acc: 0.697 - 0s 22ms/step - loss: 0.9542 - acc: 0.704 - 0s 22ms/step - loss: 0.9283 - acc: 0.711 - 0s 22ms/step - loss: 0.9589 - acc: 0.704 - 0s 22ms/step - loss: 0.9753 - acc: 0.693 - 0s 22ms/step - loss: 0.9836 - acc: 0.690 - 0s 22ms/step - loss: 0.9935 - acc: 0.686 - 0s 22ms/step - loss: 1.0086 - acc: 0.686 - 0s 22ms/step - loss: 1.0153 - acc: 0.683 - 0s 22ms/step - loss: 1.0138 - acc: 0.681 - 0s 22ms/step - loss: 1.0144 - acc: 0.680 - 0s 22ms/step - loss: 1.0132 - acc: 0.678 - 0s 22ms/step - loss: 0.9983 - acc: 0.678 - 0s 22ms/step - loss: 0.9938 - acc: 0.683 - 1s 22ms/step - loss: 1.0101 - acc: 0

[1.0342283791797175, 0.6644]

In [15]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))

In [16]:
m.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)             

In [17]:
m.evaluate(test_data)

    157/Unknown - 0s 131ms/step - loss: 0.8724 - acc: 0.73 - 0s 74ms/step - loss: 0.8360 - acc: 0.7656 - 0s 56ms/step - loss: 0.8636 - acc: 0.750 - 0s 47ms/step - loss: 0.8193 - acc: 0.757 - 0s 42ms/step - loss: 0.7991 - acc: 0.756 - 0s 38ms/step - loss: 0.7849 - acc: 0.760 - 0s 36ms/step - loss: 0.7737 - acc: 0.758 - 0s 34ms/step - loss: 0.7874 - acc: 0.753 - 0s 33ms/step - loss: 0.7934 - acc: 0.744 - 0s 32ms/step - loss: 0.7921 - acc: 0.743 - 0s 31ms/step - loss: 0.7750 - acc: 0.750 - 0s 30ms/step - loss: 0.7970 - acc: 0.740 - 0s 29ms/step - loss: 0.8038 - acc: 0.735 - 0s 29ms/step - loss: 0.8112 - acc: 0.732 - 0s 28ms/step - loss: 0.8115 - acc: 0.732 - 0s 28ms/step - loss: 0.8227 - acc: 0.731 - 0s 28ms/step - loss: 0.8359 - acc: 0.725 - 0s 27ms/step - loss: 0.8390 - acc: 0.723 - 1s 27ms/step - loss: 0.8375 - acc: 0.720 - 1s 27ms/step - loss: 0.8365 - acc: 0.720 - 1s 27ms/step - loss: 0.8197 - acc: 0.726 - 1s 26ms/step - loss: 0.8128 - acc: 0.729 - 1s 26ms/step - loss: 0.8203 - acc: 

[0.8221499644646979, 0.7214]

In [18]:
m.evaluate(valid_data)

    157/Unknown - 0s 207ms/step - loss: 1.1617 - acc: 0.68 - 0s 113ms/step - loss: 0.8635 - acc: 0.74 - 0s 82ms/step - loss: 0.8534 - acc: 0.7292 - 0s 67ms/step - loss: 0.8202 - acc: 0.738 - 0s 58ms/step - loss: 0.8347 - acc: 0.734 - 0s 52ms/step - loss: 0.8541 - acc: 0.726 - 0s 47ms/step - loss: 0.8324 - acc: 0.734 - 0s 44ms/step - loss: 0.8266 - acc: 0.728 - 0s 42ms/step - loss: 0.8192 - acc: 0.730 - 0s 40ms/step - loss: 0.8173 - acc: 0.731 - 0s 38ms/step - loss: 0.8256 - acc: 0.725 - 0s 36ms/step - loss: 0.8186 - acc: 0.729 - 0s 35ms/step - loss: 0.8023 - acc: 0.730 - 0s 34ms/step - loss: 0.8019 - acc: 0.725 - 1s 33ms/step - loss: 0.8059 - acc: 0.724 - 1s 33ms/step - loss: 0.8055 - acc: 0.726 - 1s 32ms/step - loss: 0.7955 - acc: 0.731 - 1s 31ms/step - loss: 0.8013 - acc: 0.730 - 1s 31ms/step - loss: 0.7948 - acc: 0.731 - 1s 30ms/step - loss: 0.7805 - acc: 0.735 - 1s 30ms/step - loss: 0.7859 - acc: 0.730 - 1s 30ms/step - loss: 0.7877 - acc: 0.732 - 1s 29ms/step - loss: 0.7784 - acc: 

[0.8219224079779, 0.7169]